# Tokenize LibriTTS-R Mimi for target LM

For our dataset, we currently simply use the Fish Speech TTS format:
- Text-only data formatted using [ChatML](https://gist.github.com/edwardzjl/8df07c1f7140c9a3e2f48d33a8032090) as a separate sequence "above" the audio code stream
- During sections where audio is being modeled, text stream 0 predicts the first semantic token index $n$ of the 8 Mimi residual codes as special token `<|semantic:n|>`
- For audio, "semantic" (neural, there's not a strong distinction between) codes (from Mimi) padded with 0s during text sections

It's possible this tokenization strategy can be improved, e.g. in [Defossez et al. 2024](https://arxiv.org/html/2410.00037v2#S3.SS4.SSS4) with the base transformer predicting the Whisper-timestamped word timings as an "inner monologue" and a delay between codebook timesteps. lol i'll do it later

In [1]:
from datasets import load_dataset, DatasetDict, concatenate_datasets

# If creating the libritts dataset for the first time
# from datasets import load_from_disk 
# dataset = load_from_disk("encoded_dataset")
# train_clean_100 = load_from_disk("encoded_libritts/train.clean.100/")
# train_clean_360 = load_from_disk("encoded_libritts/train.clean.360/")
# dev_clean = load_from_disk("encoded_libritts/dev.clean")
# test_clean = load_from_disk("encoded_libritts/test.clean")
# full_train = concatenate_datasets([train_clean_100, train_clean_360])
dataset = load_dataset("jkeisling/libritts-r-mimi-kokoro")
full_train = concatenate_datasets([dataset["train.clean.100"], dataset["train.clean.360"]])

dataset = DatasetDict({
    "train": full_train,
    "val": dataset["dev.clean"],
    "test": dataset["test.clean"]
})
dataset = dataset.with_format("torch")
dataset = dataset.remove_columns(["chapter_id", "text_original"])
dataset = dataset.rename_column(original_column_name="text_normalized", new_column_name="normalized_text")

/home/ritsuko/ai/audio/dual-ar/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**NOTE! This is PATH DEPENDENT on ADDING THE SEMANTIC TOKENS TO THE TOKENIZER EARLIER using `create_bytelevel_init.ipynb`. DO NOT SKIP THIS STEP OR THE MODEL WILL BE IRRETRIEVABLY BROKEN! YOU HAVE BEEN WARNED.**

==**THIS IS BYTE LEVEL!**==

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("../checkpoints/smoltts_byte_kokoro")
tokenizer.use_default_system_prompt = False

Check this carefully: for byte level, it should be 256.

In [3]:
len(tokenizer), tokenizer.vocab_size

(2368, 256)

Please manually verify the text is done correctly. However, DECODE will not work.

In [4]:
# Test the tokenizer by encoding and decoding some example text
example_text = "<|im_start|>system\n<|american|><|male|><|im_end|>"
encoded = tokenizer(example_text, return_tensors="pt")
print(f"Encoded: {encoded['input_ids']}")
decoded = tokenizer.decode(encoded['input_ids'][0])

# Print the results
decoded

Encoded: tensor([[269, 256,  10, 260, 261, 270]])


'<|im_start|>system\n<|american|><|male|><|im_end|>'

In [5]:
sequence = tokenizer.apply_chat_template([{"role": "user", "content": "help me i am trapped in this computer"}], add_generation_prompt=True,  return_tensors="pt")
sequence

tensor([[269, 257,  10, 104, 101, 108, 112,  32, 109, 101,  32, 105,  32,  97,
         109,  32, 116, 114,  97, 112, 112, 101, 100,  32, 105, 110,  32, 116,
         104, 105, 115,  32,  99, 111, 109, 112, 117, 116, 101, 114, 270,  10,
         269, 258,  10]])

In [6]:
tokenizer.decode(sequence[0, :])

'<|im_start|>user\nhelp me i am trapped in this computer<|im_end|>\n<|im_start|>assistant\n'

In [7]:
import torch

def encode_text(role: str, content: str, add_generation_prompt: bool = True) -> torch.Tensor:
    # baseline = tokenizer.apply_chat_template(f"{chr(10) if ''}<|im_start|>{role}\n{content}<|im_end|>\n",)
    baseline = tokenizer.apply_chat_template(
        [{"role": role, "content": content}],
        add_generation_prompt=add_generation_prompt,
        return_tensors="pt"
    )
    zeros_mask = torch.zeros(8, baseline.size(1), dtype=baseline.dtype)
    return torch.cat([baseline, zeros_mask])

tts_sysprompt = encode_text("system", "<|speaker:40|>", add_generation_prompt=False)
tokenizer.decode(tts_sysprompt[0,:])

'<|im_start|>system\n<|speaker:40|><|im_end|>\n'

Note that this assumes you're using ChatML. if you're NOT, then there's quite a bit more to fix.

In [13]:
SEMANTIC_OFFSET = tokenizer.encode("<|semantic:0|>")[0]
# B * C+1 * 2
VQ_USER_PREFIX = encode_text(role="user", content="")[:,:-2]
TRAILING_IM_END = torch.tensor([
    tokenizer.encode("<|im_end|>") + [0] * 8,
    tokenizer.encode("\n") + [0] * 8,
]).T

def encode_vq(codes: torch.Tensor, is_assistant=True) -> torch.Tensor:
    """
    Expects C * T
    """
    if codes.ndim != 2:
        raise ValueError("Must be single batch")
    speaker_line = codes[0,:] + SEMANTIC_OFFSET
    vq_block = torch.cat([speaker_line.unsqueeze(0), codes])

    block = torch.cat([vq_block, TRAILING_IM_END], dim=1)
    return block if is_assistant else torch.cat([VQ_USER_PREFIX, block], dim=1)


out = encode_vq(dataset["test"][0]["codes"], is_assistant=True)
tokenizer.decode(out[0,:])

'<|semantic:1049|><|semantic:127|><|semantic:1880|><|semantic:1031|><|semantic:1031|><|semantic:1031|><|semantic:1799|><|semantic:1268|><|semantic:549|><|semantic:1768|><|semantic:184|><|semantic:1538|><|semantic:95|><|semantic:648|><|semantic:1281|><|semantic:162|><|semantic:1782|><|semantic:536|><|semantic:230|><|semantic:2040|><|semantic:133|><|semantic:1117|><|semantic:674|><|semantic:507|><|semantic:507|><|semantic:1399|><|semantic:640|><|semantic:1277|><|semantic:330|><|semantic:740|><|semantic:690|><|semantic:875|><|semantic:851|><|semantic:1772|><|semantic:186|><|semantic:670|><|semantic:1612|><|semantic:1532|><|semantic:119|><|semantic:1052|><|semantic:1029|><|semantic:1029|><|semantic:1532|><|semantic:1161|><|semantic:1344|><|semantic:1109|><|semantic:544|><|semantic:6|><|semantic:56|><|semantic:1640|><|semantic:1928|><|semantic:1968|><|semantic:657|><|semantic:382|><|semantic:382|><|semantic:596|><|semantic:553|><|semantic:2030|><|semantic:578|><|semantic:531|><|semantic:616

In [14]:
speaker_names = ["default", "bella", "nicole", "sarah", "sky", "adam", "michael", "emma", "isabella", "george", "lewis"]
speaker_ids = {value: index for index, value in enumerate(speaker_names)}
speaker_ids["adam"]

4

In [25]:
from typing import Dict
import random

# TODO: Not doing ASR for now
def tts_tokenize_row(row: Dict):
    """
    NOTE: Deliberately ignores sysprompt line for now, can be done in packing
    """
    # TODO: Fix this upstream in the data gen!
    gender = "<|male|>" if row["speaker_id"] in ["george", "lewis", "adam", "michael"] else "<|female|>"
    accent = f"<|{row['accent']}|>"
    speaker = f"<|speaker:{speaker_ids[row['speaker_id']]}|>" if random.random() < 0.7 else ""

    # Just keep it all for now, will test generalization later
    system_line = encode_text(role="system", content="".join([accent, gender, speaker]))
    user_line = encode_text(
        role="user", 
        content=row["normalized_text"].encode("utf-8").decode("latin-1"), 
        add_generation_prompt=True
    )
    assistant_line = encode_vq(row["codes"])
    ground_truth = torch.cat([system_line, user_line, assistant_line], dim=1)
    # Causal shift
    tokens = ground_truth[:,:-1].clone()
    labels = ground_truth[:,1:].clone()

    # Assuming user line took care of assistant prefix
    labels[1:, :user_line.size(1) - 1] = -100
    # Mask out newline
    labels[1:, -1] = -100

    return({
        "tokens": tokens,
        "labels": labels
    })
    


example_row = tts_tokenize_row(dataset["test"][10])
tokenizer.decode(example_row["labels"][0,:])

'system\n<|american|><|female|><|im_end|>\n<|im_start|>assistant\n<|im_start|>user\nYou haven\'t spoken a word."<|im_end|>\n<|im_start|>assistant\n<|semantic:1049|><|semantic:127|><|semantic:1880|><|semantic:1031|><|semantic:1031|><|semantic:1031|><|semantic:1492|><|semantic:1492|><|semantic:1923|><|semantic:1488|><|semantic:376|><|semantic:1849|><|semantic:1906|><|semantic:1135|><|semantic:334|><|semantic:1782|><|semantic:1399|><|semantic:808|><|semantic:689|><|semantic:997|><|semantic:168|><|semantic:1702|><|semantic:2009|><|semantic:56|><|semantic:769|><|semantic:666|><|semantic:84|><|semantic:84|><|semantic:752|><|semantic:752|><|semantic:1850|><|semantic:752|><|semantic:481|><|semantic:481|><|im_end|>\n'

In [26]:
dataset["test"][3]

{'normalized_text': "It's abominable of you to ask me.",
 'speaker_id': 'default',
 'id': '4446_2275_000042_000004',
 'codes': tensor([[1049,  127, 1880, 1031, 1031, 1031, 1492, 1492,  322,  301, 1237,  680,
          1477, 1275, 2003,   22, 1137, 1137, 1347, 1293,  786,   70,  336, 1323,
          1508, 1421, 1215, 1424,  779,  779,  705,  582,  666,  716,   84,   84,
           752,  752,  752,  752,  752],
         [1700,  243,  243,  243,  243, 1211, 1211, 1211, 1469,  678,  106,  393,
          1032, 1377,  359,  457, 1114, 1859, 1196,  653,   56, 1731,  352,  375,
          1295, 1847,   96,  446,  724,  317, 1966, 1700,  243,  243,  243,  243,
           243,  243,  243,  243,  243],
         [1626,  783, 1178, 1178, 1178, 1697, 1697, 1697, 1591, 1221, 2025, 1302,
           600, 1294, 1214,  790,   38, 1890, 2040,  738, 1298, 1672,  713,  360,
          1739,   87, 1627,  710,  466, 1063, 1039, 1029,  783, 1178,  783, 1178,
          1178, 1178, 1178,  783, 1178],
         [ 54

In [27]:
example_row["labels"]

tensor([[ 256,   10,  260,  262,  270,   10,  269,  258,   10,  269,  257,   10,
           89,  111,  117,   32,  104,   97,  118,  101,  110,   39,  116,   32,
          115,  112,  111,  107,  101,  110,   32,   97,   32,  119,  111,  114,
          100,   46,   34,  270,   10,  269,  258,   10, 1369,  447, 2200, 1351,
         1351, 1351, 1812, 1812, 2243, 1808,  696, 2169, 2226, 1455,  654, 2102,
         1719, 1128, 1009, 1317,  488, 2022, 2329,  376, 1089,  986,  404,  404,
         1072, 1072, 2170, 1072,  801,  801,  270,   10],
        [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0, 1049,  127, 1880, 1031,
         1031, 1031, 1492, 1492, 1923, 1488,  376, 1849, 1906, 1135,  334, 1782,
         1399,  808,  689,  997,  168, 1702, 2009, 

In [28]:
# DO NOT INCREASE batch size
dataset = dataset.map(tts_tokenize_row, remove_columns="codes")

Map: 100%|██████████| 4837/4837 [00:03<00:00, 1277.28 examples/s]


In [29]:
dataset.save_to_disk("tokenized_libritts_bytes_kokoro_speaker")

Saving the dataset (1/1 shards): 100%|██████████| 4837/4837 [00:00<00:00, 61758.11 examples/s]


In [12]:
NEWLINE_SEPARATOR = torch.tensor(tokenizer.encode("\n") + [0] * 8).unsqueeze(1)

def batch_pack_sequences(examples, window_size=768, max_items=5):
   """
   Pack sequences with system prompt and metrics
   """
   packed_tokens = []
   packed_labels = []
   packed_speakers = []
   pack_lengths = []
   items_per_pack = []
   
   tokens = examples['tokens']
   labels = examples['labels']
   speakers = examples['speaker_id']
   
   # Account for system prompt in window size
   effective_window = window_size - tts_sysprompt.shape[1]
   
   for i in range(len(tokens)):
       seq_len = tokens[i].shape[1]
       
       # Start new pack
       if i == 0 or current_length + seq_len > effective_window or \
          current_speaker != speakers[i] or current_items >= max_items:
           
           # Save previous pack if it exists
           if i > 0 and current_tokens:
               packed_tokens.append(torch.cat(current_tokens, dim=1))
               packed_labels.append(torch.cat(current_labels, dim=1))
               packed_speakers.append(current_speaker)
               pack_lengths.append(current_length + tts_sysprompt.shape[1])
               items_per_pack.append(current_items)
           
           # Initialize new pack with system prompt
           current_tokens = [tts_sysprompt, tokens[i]]
           current_labels = [tts_sysprompt, labels[i]]
           current_speaker = speakers[i]
           current_length = seq_len
           current_items = 1
           continue
           
       # Add to current pack with separator
       current_tokens.extend([NEWLINE_SEPARATOR, tokens[i]])
       current_labels.extend([NEWLINE_SEPARATOR, labels[i]])
       current_length += seq_len + 1
       current_items += 1
   
   # Don't forget last pack
   if current_tokens:
       packed_tokens.append(torch.cat(current_tokens, dim=1))
       packed_labels.append(torch.cat(current_labels, dim=1))
       packed_speakers.append(current_speaker)
       pack_lengths.append(current_length + tts_sysprompt.shape[1])
       items_per_pack.append(current_items)
   
   return {
       'tokens': packed_tokens,
       'labels': packed_labels,
       'speaker_id': packed_speakers,
       'pack_length': pack_lengths,
       'items_in_pack': items_per_pack
   }

In [16]:
# Usage:
packed_dataset = dataset.map(
    lambda row: batch_pack_sequences(row, max_items=3),
    batched=True,
    remove_columns=dataset['val'].column_names,
    batch_size=1000  # Adjust based on memory constraints
)

Map: 100%|██████████| 4837/4837 [00:01<00:00, 4760.96 examples/s]


In [14]:
example_row = packed_dataset['val'][0]
tokenizer.decode(example_row["tokens"][0,:])

'<|im_start|>system\nSpeak out the provided text<|im_end|>\n<|im_start|>assistant\n<|im_start|>user\nThe weapon must still have been there.<|im_end|>\n<|im_start|>assistant\n<|semantic:1049|><|semantic:1114|><|semantic:1609|><|semantic:784|><|semantic:499|><|semantic:260|><|semantic:1011|><|semantic:8|><|semantic:1407|><|semantic:540|><|semantic:1615|><|semantic:561|><|semantic:1945|><|semantic:201|><|semantic:1324|><|semantic:668|><|semantic:376|><|semantic:1849|><|semantic:9|><|semantic:1921|><|semantic:1921|><|semantic:1683|><|semantic:228|><|semantic:897|><|semantic:1677|><|semantic:518|><|im_end|>\n<|im_start|>user\nHow quickly he disappeared!"<|im_end|>\n<|im_start|>assistant\n<|semantic:1698|><|semantic:1848|><|semantic:1021|><|semantic:414|><|semantic:972|><|semantic:1252|><|semantic:1545|><|semantic:1363|><|semantic:307|><|semantic:722|><|semantic:1169|><|semantic:170|><|semantic:1701|><|semantic:1967|><|semantic:886|><|semantic:1540|><|semantic:1540|><|semantic:1113|><|semant

In [17]:
packed_dataset.save_to_disk("tokenized_libritts_packed_3")

Saving the dataset (1/1 shards): 100%|██████████| 1649/1649 [00:00<00:00, 31396.69 examples/s]


## Appendix: Markdown

In [24]:
tokenizer.decode(example_row[0]["labels"][0,:])

'system\nSpeak out the provided text<|im_end|>\n<|im_start|>user\nI felt it in my bones when I woke this morning that something splendid was going to turn up.<|im_end|>\n<|im_start|>assistant\n<|semantic:1049|><|semantic:1268|><|semantic:549|><|semantic:1324|><|semantic:668|><|semantic:1538|><|semantic:1593|><|semantic:95|><|semantic:629|><|semantic:1281|><|semantic:1281|><|semantic:680|><|semantic:536|><|semantic:536|><|semantic:230|><|semantic:1018|><|semantic:1117|><|semantic:244|><|semantic:507|><|semantic:997|><|semantic:1399|><|semantic:640|><|semantic:1591|><|semantic:1967|><|semantic:1161|><|semantic:690|><|semantic:67|><|semantic:1772|><|semantic:830|><|semantic:1612|><|semantic:561|><|semantic:119|><|semantic:1052|><|semantic:880|><|semantic:1029|><|semantic:1532|><|semantic:1161|><|semantic:1344|><|semantic:1109|><|semantic:6|><|semantic:1001|><|semantic:382|><|semantic:596|><|semantic:99|><|semantic:1726|><|semantic:2030|><|semantic:531|><|semantic:616|><|semantic:367|><|se

In [25]:
tokenizer.decode(example_row[1]["labels"][0,:])

'system\nTranscribe the provided speech<|im_end|>\n<|im_start|>user\n<|semantic:1049|><|semantic:1268|><|semantic:549|><|semantic:1324|><|semantic:668|><|semantic:1538|><|semantic:1593|><|semantic:95|><|semantic:629|><|semantic:1281|><|semantic:1281|><|semantic:680|><|semantic:536|><|semantic:536|><|semantic:230|><|semantic:1018|><|semantic:1117|><|semantic:244|><|semantic:507|><|semantic:997|><|semantic:1399|><|semantic:640|><|semantic:1591|><|semantic:1967|><|semantic:1161|><|semantic:690|><|semantic:67|><|semantic:1772|><|semantic:830|><|semantic:1612|><|semantic:561|><|semantic:119|><|semantic:1052|><|semantic:880|><|semantic:1029|><|semantic:1532|><|semantic:1161|><|semantic:1344|><|semantic:1109|><|semantic:6|><|semantic:1001|><|semantic:382|><|semantic:596|><|semantic:99|><|semantic:1726|><|semantic:2030|><|semantic:531|><|semantic:616|><|semantic:367|><|semantic:1271|><|semantic:1868|><|semantic:978|><|semantic:729|><|semantic:396|><|semantic:1544|><|im_end|>\n<|im_start|>assis

In [21]:
example_row["tokens"][0][1,:]

tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 1698, 1719,  204, 1389,  851, 1772,  186, 1307, 1895,  832,
        1633,  771,  648, 1530, 1989, 1574, 1348,  722,  144, 1945,  278, 1109,
          29,  611,   46,  622,  628, 1740,  572,  572,  345, 1989, 1676,  929,
        1776,  749,  313, 1997, 1571,  819, 1238, 1054, 1054, 1135, 1506, 1393,
         616, 1702,  993,  579,  486,  486, 2039,  148,  657,  664,  339,  339,
         588,  212, 1443,   32, 1320, 1549,  440,    8, 1407, 1722, 1650, 1615,
         798,  121,  303,  697,  837,  358, 1882,  440, 1992, 1992,  587,  178,
         178, 1627, 1530,  929, 1610, 1916,  523,  213, 1252, 1480, 1468, 1899,
         773, 2033, 2033,   83, 1146,  7

In [28]:
example_row[1]["labels"][1,:]

tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, 1049, 1268,  549, 1324,  668, 1538, 1593,   95,  629, 1281, 1281,
         680,  536,  536,  230, 1018, 1117,  244,  507,  997, 1399,  640, 1591,
        1967, 1161,  690,   67, 1772,  830, 1612,  561,  119, 1052,  880, 1029,
        1532, 1161, 1344, 1109,    6, 1001,  382,  596,   99, 1726, 2030,  531,
         616,  367, 1271, 1868,  978,  729,  396, 1544,    0, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100])

In [28]:
dataset["test"][0]

{'normalized_text': 'I felt it in my bones when I woke this morning that something splendid was going to turn up.',
 'speaker_id': '4446',
 'id': '4446_2275_000002_000009',
 'tokens': tensor([[    1,  9690,   198, 15024,   494,   578,   260,  2711,  1694,     2,
            198,     1,  4093,   198,    57,  4592,   357,   281,   957,  6542,
            645,   339, 40652,   451,  5738,   338,  1488, 33494,   436,  2045,
            288,  1607,   614,    30,     2,   198,     1,   520,  9531,   198,
          50201, 50420, 49701, 50476, 49820, 50690, 50745, 49247, 49781, 50433,
          50433, 49832, 49688, 49688, 49382, 50170, 50269, 49396, 49659, 50149,
          50551, 49792, 50743, 51119, 50313, 49842, 49219, 50924, 49982, 50764,
          49713, 49271, 50204, 50032, 50181, 50684, 50313, 50496, 50261, 49158,
          50153, 49534, 49748, 49251, 50878, 51182, 49683, 49768, 49519, 50423,
          51020, 50130, 49881, 49548, 50696],
         [    0,     0,     0,     0,     0,     0,